In [1]:
# Import the libraries
%matplotlib inline
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from mpl_toolkits.mplot3d import Axes3D 
from sklearn import linear_model
import seaborn as sns
from statsmodels.graphics.gofplots import ProbPlot
from statsmodels.compat import lzip
import statsmodels.stats.api as sms
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm 
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from pandas import DataFrame
import math
import numpy
from statistics import *
from numpy import nan

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Read the race file
from google.colab import drive
drive.mount("/content/drive/")
%cd '/content/drive/My Drive/DS440/Income_and_race'
!pwd #Prints the present working directory
racedata = pd.read_csv('race.csv')


Mounted at /content/drive/
/content/drive/My Drive/DS440/Income_and_race
/content/drive/My Drive/DS440/Income_and_race


,census_block_group,total.population,total.population.ME,race.white.alone.estimate,race.white.alone.ME,african.american.alone.estimate,african.american.alone.ME,american.indian.alaskan.native.estimate,american.indian.alaskan.native.ME,asian.estimate,asian.ME,native.hawaiian.other.islander.estimate,native.hawaiian.other.islander.ME,other.race.alone,other.race.ME,two.or.more.estimate,two.or.more.ME,Two.races.excluding.Some.other.race.estimate,Two.races.excluding.Some.other.race.ME,Two.races.excluding.Some.other.race.and.three.or.more.races.estimate,Two.races.excluding.Some.other.race.and.three.or.more.races.ME
0,10010201001,745,226,585,198,160,140,0,11,0,11,0,11,0,11,0,11,0,11,0,11
1,10010201002,1265,232,1083,222,104,75,0,11,9,13,0,11,0,11,69,46,0,11,69,46
2,10010202001,960,261,361,179,568,195,0,11,0,11,13,20,0,11,18,30,0,11,18,30
3,10010202002,1236,227,615,154,571,168,0,11,24,36,0,11,10,12,16,24,0,11,16,24
4,10010203001,2364,378,1481,314,515,197,22,34,27,32,6,14,228,285,85,103,0,11,85,103


In [3]:
# Read the race data
minorityportion = 1-(racedata['race.white.alone.estimate'].astype(float)/racedata['total.population'].astype(float))
minorityportion = minorityportion.astype(float)
racedata["minorityportion"]=minorityportion

,census_block_group,total.population,total.population.ME,race.white.alone.estimate,race.white.alone.ME,african.american.alone.estimate,african.american.alone.ME,american.indian.alaskan.native.estimate,american.indian.alaskan.native.ME,asian.estimate,asian.ME,native.hawaiian.other.islander.estimate,native.hawaiian.other.islander.ME,other.race.alone,other.race.ME,two.or.more.estimate,two.or.more.ME,Two.races.excluding.Some.other.race.estimate,Two.races.excluding.Some.other.race.ME,Two.races.excluding.Some.other.race.and.three.or.more.races.estimate,Two.races.excluding.Some.other.race.and.three.or.more.races.ME,minorityportion
0,10010201001,745,226,585,198,160,140,0,11,0,11,0,11,0,11,0,11,0,11,0,11,0.214765
1,10010201002,1265,232,1083,222,104,75,0,11,9,13,0,11,0,11,69,46,0,11,69,46,0.143874
2,10010202001,960,261,361,179,568,195,0,11,0,11,13,20,0,11,18,30,0,11,18,30,0.623958
3,10010202002,1236,227,615,154,571,168,0,11,24,36,0,11,10,12,16,24,0,11,16,24,0.502427
4,10010203001,2364,378,1481,314,515,197,22,34,27,32,6,14,228,285,85,103,0,11,85,103,0.373519


In [6]:
# Read income data 
!pwd #Prints the present working directory
incomedata = pd.read_csv('income.csv')
incomedata.head()

/content/drive/My Drive/DS440/Income_and_race


,census_block_group,total.households,households.ME,less.than.10000,less.than.1000.ME,10000.to.14999,10000.to.14999.ME,15000.to.19999,15000.to.19999.ME,20000.to.24999,20000.to.24999.ME,25000.to.29999,25000.to.29999.ME,30000.to.34999,30000.to.34999.ME,35000.to.39999,35000.to.39999.ME,40000.to.44999,40000.to.44999.ME,45000.to.49999,45000.to.49999.ME,50000.to.59999,50000.to.59999.ME,60000.to.74999,60000.to.74999.ME,75000.to.99999,75000.to.99999.ME,10000.to.124999,10000.to.124999.ME,125000.to.149999,125000.to.149999.ME,150000.to.199999,150000.to.199999.ME,200000.or.more,200000.or.more.ME,MEDIAN.INCOME,MEDIAN.INCOME.ME
0,10010201001,284,53,24,29,33,25,23,22,25,20,39,49,5,9,0,11,10,11,0,11,12,12,43,31,35,32,6,9,29,27,0,11,0,11,NaN,NaN
1,10010201002,456,64,8,10,15,24,14,22,5,8,10,11,31,30,25,27,8,9,19,16,25,28,57,32,101,50,47,26,27,23,55,31,9,11,77813.0,11618.0
2,10010202001,386,78,47,39,108,47,37,31,0,11,23,27,4,9,13,13,20,27,0,11,4,7,28,21,89,47,9,13,4,7,0,11,0,11,25179.0,20248.0
3,10010202002,452,85,32,27,37,30,15,15,18,19,12,16,28,33,14,16,69,71,43,28,25,22,16,17,44,34,56,30,43,28,0,11,0,11,45104.0,6377.0
4,10010203001,824,108,23,24,34,50,16,18,17,19,47,38,44,39,51,45,33,29,26,29,203,96,121,55,82,50,87,48,18,21,22,20,0,11,55222.0,2034.0


In [7]:
# impute median income data for census blocks with missing income data

missing_income_data=incomedata[incomedata["MEDIAN.INCOME"].isna()]
missing_income_data=missing_income_data[["census_block_group", "total.households", "less.than.10000", "10000.to.14999", "15000.to.19999", "20000.to.24999", "25000.to.29999", "30000.to.34999"
, "35000.to.39999", "40000.to.44999", "45000.to.49999", "50000.to.59999", "60000.to.74999", "75000.to.99999", "10000.to.124999", "125000.to.149999", "150000.to.199999", "200000.or.more"]]
missing_income_data = missing_income_data[missing_income_data['total.households'] != 0]
missing_income_data.head()
median=(missing_income_data["total.households"]+1)/2
flooring=numpy.floor(median)
missing_income_data["median"]=flooring
missing_income_data_temp=missing_income_data.drop(columns=["census_block_group", "total.households", "median"])
the_list=(missing_income_data_temp.values.tolist())
the_cum_list=[]
for i in range(0, len(the_list)):
  a_list=the_list[i]
  x=np.cumsum(a_list)
  the_cum_list.append(list(x))
missing_income_data.head()

pos=[]
check=[]
flooring_interest=list(flooring)

for i in range(0, len(the_cum_list)):
  #print(i)
  a_list=the_cum_list[i]
  interest=flooring_interest[i]
  #print(a_list)
  for n in range(0, len(a_list)):
    if (a_list[n]) < interest:
      continue
    else:
      #print(a_list)
      pos.append(n)
      check.append(a_list)
      break
len(check)


#type(missing_income_data["the_list"])

pred_median=[]
for i in range(0, len(pos)):
  x=pos[i]
  if x==1:
    y=9999/2
  if x==2:
    y=(10000+ 14999)/2
  if x==3:
    y=(15000+ 19999)/2
  if x==4:
    y=(20000+ 24999)/2
  if x==5:
    y=(25000+ 29999)/2
  if x==6:
    y=(30000+ 34999)/2
  if x==7:
    y=(35000+ 39999)/2
  if x==8:
    y=(40000+ 44999)/2
  if x==9:
    y=(45000+49999)/2
  if x==10:
    y=(50000+ 59999)/2
  if x==11:
    y=(60000+ 74999)/2
  if x==12:
    y=(75000+ 99999)/2
  if x==13:
    y=(100000+ 124999)/2
  if x==14: 
    y=(125000+ 149999)/2
  if x==15:
    y=(150000+ 199999)/2
  if x==16:
    y=200000
  pred_median.append(y)
len(pred_median)
missing_income_data["MEDIAN.INCOME.PRED"]=pred_median
missing_income_data.head()
missing_income_data_total = pd.merge(incomedata, missing_income_data, how="left", on=["census_block_group"])
# missing_income_data_total.shape
final_list=[]
x=list(incomedata["MEDIAN.INCOME"])
x = [str(l) for l in x]
y=list(missing_income_data_total["MEDIAN.INCOME.PRED"])
y = [str(l) for l in y]

for i in range(0,len(x)):
  income=x[i]
  if income==nan or income=="nan":
    median=y[i]
    final_list.append(median)
  else:
    median=income
    final_list.append(median)

final_list = [float(l) for l in final_list]
incomedata["MEDIAN.INCOME"]=final_list
result = pd.merge(incomedata, racedata, how="inner", on=["census_block_group"])
result = result[result['total.population'] != 0]
relevant_data = result[["census_block_group", "minorityportion", "MEDIAN.INCOME", "total.population"]]
relevant_data.head()
population=relevant_data["total.population"]
census_block_classification=[]
len(list(population))

,census_block_group,total.households,less.than.10000,10000.to.14999,15000.to.19999,20000.to.24999,25000.to.29999,30000.to.34999,35000.to.39999,40000.to.44999,45000.to.49999,50000.to.59999,60000.to.74999,75000.to.99999,10000.to.124999,125000.to.149999,150000.to.199999,200000.or.more,median
0,10010201001,284,24,33,23,25,39,5,0,10,0,12,43,35,6,29,0,0,142.0
88,10030112024,326,0,21,10,27,0,61,0,59,0,11,0,43,0,12,0,82,163.0
104,10030114062,318,0,42,52,25,16,19,9,10,9,9,0,15,64,0,40,8,159.0
283,10150020005,430,13,35,83,15,13,0,33,24,0,0,28,96,90,0,0,0,215.0
313,10159819011,4,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,2.0


In [24]:
# Classify rural or urban based on census definition
def census_classification(population):
  for i in range(0,len(population)):
    int_population = (list(population)[i])
    if int_population<2500:
      classification="Rural"
      census_block_classification.append(classification)
    elif int_population>=50000:
      classification="Urbanized Area"
      census_block_classification.append(classification)
    else:
      classification="Urban Clusters"
      census_block_classification.append(classification)
  return census_block_classification

census_block_classification=census_classification(population)
relevant_data["census_block_classification"]=census_block_classification

In [28]:
# write the data
relevant_data.to_csv('relevant.data.csv')  
result["census_block_classification"]=census_block_classification
result.to_csv('interesting_data.csv')